In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import torch
import torchvision
from torchvision import datasets, models
from torchvision.transforms import functional as FT
from torchvision import transforms as T
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split, Dataset
import copy
import math
from PIL import ImageTk, Image
import cv2
import albumentations as A  # our data augmentation library


import tkinter as tk
from tkinter import ttk, Frame, Button, Canvas, Label
from tkinter.filedialog import askopenfilename
import sys
import matplotlib.pyplot as plt


C:\Users\krusl\anaconda3\envs\project2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = ""

def resizer(url):
    imagg = Image.open(url)
    resized_imagg = imagg.resize((1230, 820), Image.ANTIALIAS)
    imagg = ImageTk.PhotoImage(resized_imagg)
    return imagg

def b1():
    panel.delete("all")
    filename = askopenfilename()
    img = resizer(filename)
    panel.create_image(615, 410, anchor="c", image=img)
    panel.image = img

#объявление объектов
models = ["FasterRCNN", "YOLO"]
window = tk.Tk()
window.title("Добро пожаловать")
window.minsize(1400, 900)
window.config(bg="skyblue")

left_frame = Frame(window, width=150, height= 820)
right_frame = Frame(window, width=1250, height=820)
right_frame.grid(row=0, column=1, padx=10, pady=5)
left_frame.grid(row=0, column=0, padx=10, pady=5)


btn1 = Button(left_frame, text='Выбрать картинку', command=b1)
btn1.grid(row=2, column=0, padx=20, pady=20)
btn2 = Button(left_frame, text= 'Запустить')
btn2.grid(row=4, column=0, padx=20, pady=20)
l = Label(left_frame, text = "Выбор модели")
l.grid(row=0, column=0, padx=20, pady=20)
combobox = ttk.Combobox(left_frame, values=models)
combobox.grid(row=1, column=0, padx=20, pady=20)

panel = Canvas(right_frame, width = 1230, height=820)
panel.pack(fill="both", expand=True)
img = resizer("C:/Users/krusl/Downloads/FasterRCNN2/gray.jpg")
panel.create_image(615, 410, anchor="c", image=img)
panel.grid()


    

window.mainloop()

C:\Users\krusl\AppData\Local\Temp\ipykernel_11444\2474870303.py:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resized_imagg = imagg.resize((1230, 820), Image.ANTIALIAS)
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\krusl\anaconda3\envs\project2\lib\site-packages\PIL\Image.py", line 3096, in open
    fp.seek(0)
AttributeError: 'str' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\krusl\anaconda3\envs\project2\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\krusl\AppData\Local\Temp\ipykernel_11444\2474870303.py", line 12, in b1
    img = resizer(filename)
  File "C:\Users\krusl\AppData\Local\Temp\ipykernel_11444\2474870303.py", line 4, in resizer
    imagg = Image.open(url)
  File "C:\Users\krusl\anaconda3\envs\project2\lib

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict, deque
import datetime
import time
from tqdm import tqdm # progress bar
from torchvision.utils import draw_bounding_boxes

!pip install pycocotools
from pycocotools.coco import COCO
from albumentations.pytorch import ToTensorV2

In [ ]:
def get_transforms(train=False):
    if train:
        transform = A.Compose([
            A.Resize(270, 480), # our input size can be 600px
            A.HorizontalFlip(p=0.3),
            A.VerticalFlip(p=0.3),
            A.RandomBrightnessContrast(p=0.1),
            A.ColorJitter(p=0.1),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco'))
    else:
        transform = A.Compose([
            A.Resize(270, 480), # our input size can be 600px
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco'))
    return transform

In [ ]:
class AquariumDetection(datasets.VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None, transforms=None):
        # the 3 transform parameters are reuqired for datasets.VisionDataset
        super().__init__(root, transforms, transform, target_transform)
        self.split = split #train, valid, test
        self.coco = COCO(os.path.join(root, split, "_annotations.coco.json")) # annotatiosn stored here
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.ids = [id for id in self.ids if (len(self._load_target(id)) > 0)]
    
    def _load_image(self, id: int):
        path = self.coco.loadImgs(id)[0]['file_name']
        image = cv2.imread(os.path.join(self.root, self.split, path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image
    def _load_target(self, id):
        return self.coco.loadAnns(self.coco.getAnnIds(id))
    
    def __getitem__(self, index):
        id = self.ids[index]
        image = self._load_image(id)
        target = self._load_target(id)
        target = copy.deepcopy(self._load_target(id))
        
        boxes = [t['bbox'] + [t['category_id']-1] for t in target] # required annotation format for albumentations
        if self.transforms is not None:
            transformed = self.transforms(image=image, bboxes=boxes)
        
        image = transformed['image']
        boxes = transformed['bboxes']
        
        new_boxes = [] # convert from xywh to xyxy
        for box in boxes:
            xmin = box[0]
            xmax = xmin + box[2]
            ymin = box[1]
            ymax = ymin + box[3]
            new_boxes.append([xmin, ymin, xmax, ymax])
        
        boxes = torch.as_tensor(new_boxes, dtype=torch.float32)
        
        tempList = [t['image_id'] for t in target]
        tempList2 = []
        tempList2.append(tempList[0])
        targ = {} # here is our transformed target
        targ['boxes'] = boxes
        targ['labels'] = torch.as_tensor([t['category_id']-1 for t in target], dtype=torch.int64)
        targ['image_id'] = torch.tensor(tempList2)
        targ['area'] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]) # we have a different area
        targ['iscrowd'] = torch.as_tensor([t['iscrowd'] for t in target], dtype=torch.int64)
        return image.div(255), targ # scale images
    def __len__(self):
        return len(self.ids)

In [ ]:
dataset_path = "C:/Users/krusl/Downloads/FasterRCNN2"

In [ ]:
coco = COCO(os.path.join(dataset_path, "train", "_annotations.coco.json"))
categories = coco.cats
n_classes = len(categories.keys())
print(n_classes)

In [ ]:
classes = [i[1]['name'] for i in categories.items()]
print(categories)

In [ ]:
train_dataset = AquariumDetection(root=dataset_path, transforms=get_transforms(True))
valid_dataset = AquariumDetection(root=dataset_path, split='valid', transforms=get_transforms(True))
#train_dataset._load_target(1)
train_dataset[1]

In [ ]:
print(train_dataset._load_target(22))
sample = train_dataset[2]
img_int = torch.tensor(sample[0] * 255, dtype=torch.uint8)
plt.imshow(draw_bounding_boxes(
    img_int, sample[1]['boxes'], [classes[i] for i in sample[1]['labels']], width=4
).permute(1, 2, 0))

In [ ]:
model = models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features # we need to change the head
print(n_classes)
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, n_classes)
import multiprocessing

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=6, num_workers = 0,shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=6, num_workers = 0,shuffle=False, collate_fn=collate_fn)

In [ ]:
images,targets = next(iter(train_loader))

images = list(image for image in images)
targets = [{k:v for k, v in t.items()} for t in targets]
print(len(targets))
output = model(images, targets) # just make sure this runs without error

In [ ]:
device = torch.device("cuda") # use GPU to train
model = model.to(device)

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, nesterov=True, weight_decay=1e-4)

In [ ]:
import sys

In [ ]:
def my_train_one_epoch(model, optimizer, loader, device, epoch):
    model.to(device)
    model.train()
    
#     lr_scheduler = None
#     if epoch == 0:
#         warmup_factor = 1.0 / 1000 # do lr warmup
#         warmup_iters = min(1000, len(loader) - 1)
        
#         lr_scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor = warmup_factor, total_iters=warmup_iters)
    
    all_losses = []
    all_losses_dict = []
    
    for images, targets in tqdm(loader):
        images = list(image.to(device) for image in images)
        targets = [{k: torch.tensor(v).to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets) # the model computes the loss automatically if we pass in targets
        losses = sum(loss for loss in loss_dict.values())
        loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
        loss_value = losses.item()
        
        all_losses.append(loss_value)
        all_losses_dict.append(loss_dict_append)
        
        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping trainig") # train if loss becomes infinity
            print(loss_dict)
            sys.exit(1)
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
#         if lr_scheduler is not None:
#             lr_scheduler.step() # 
        
    all_losses_dict = pd.DataFrame(all_losses_dict) # for printing
    print("Epoch {}, lr: {:.6f}, loss: {:.6f}, loss_classifier: {:.6f}, loss_box: {:.6f}, loss_rpn_box: {:.6f}, loss_object: {:.6f}".format(
        epoch, optimizer.param_groups[0]['lr'], np.mean(all_losses),
        all_losses_dict['loss_classifier'].mean(),
        all_losses_dict['loss_box_reg'].mean(),
        all_losses_dict['loss_rpn_box_reg'].mean(),
        all_losses_dict['loss_objectness'].mean()
    ))

In [ ]:
num_epochs=1
from engine import train_one_epoch, evaluate
import utils

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=1000)
    evaluate(model, valid_loader, device=device)

In [ ]:
torch.cuda.empty_cache()
torch.save(model.state_dict(), "C:/Users/krusl/Downloads/FasterRCNN2/model.pt")
model.eval()

In [ ]:
path = 'C:/Users/krusl/Downloads/FasterRCNN2/test/trash2.jpg'
from torchvision.io import read_image
pic = read_image(path)
img_int = torch.tensor(pic, dtype=torch.uint8)
with torch.no_grad():
    prediction = model([pic.div(255).to(device)])
    pred = prediction[0]

In [ ]:
fig = plt.figure(figsize=(14, 10))
plt.imshow(draw_bounding_boxes(img_int,
    pred['boxes'][pred['scores'] > 0.3],
    [classes[i] for i in pred['labels'][pred['scores'] > 0.3].tolist()], width=4
).permute(1, 2, 0))